In [1]:
"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function

import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

np.random.seed(0)

import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
# from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import numpy as np
import cv2
import os
import torch

import glob
import matplotlib.pyplot as plt
import math
import sympy
from IPython.display import display
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from __future__ import print_function

import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter



C:\Users\ahmed.mansour\Anaconda3\envs\ocr_cpu\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\ahmed.mansour\Anaconda3\envs\ocr_cpu\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit 

In [2]:
# objpoints[5][0],imgpoints[5][0]
oj =  [[0,0],[91.4,0],[579.4,0],[579.4,579],[945.4,0],[945.4,579],
             [1524,0],[579.4,1300],[701.4,1300],[823.4,1300],[945.4,1300]]
# imgpo1 = [[432,288],[561,274],[596,338],[730,309]]
imgpo1 =[[106,327],[218,313],[432,288],[596,338],[561,274],[730,309],
              [705,256],[1087,492],[1116,457],[1143,427],[1159,402]]
# imgpo2 = [[349,132],[293,124],[148,169],[259,120]]
img = cv2.imread('basket0.png')
# poitdist = cv2.undistortPoints(np.float32(imgpo1),K1,dist1)
# poitdist.reshape(4,2).shape,np.float32(oj).shape
H1 = cv2.findHomography(np.float32(imgpo1),np.float32(oj),0)[0]


In [3]:
imgpo2= [[596,283],[631,289],[733,308],[554,333],[855,330],[674,368],
              [1183,392],[125,392],[135,415],[149,444],[166,482]]
# imgpo2 = [[349,132],[293,124],[148,169],[259,120]]
img = cv2.imread('basket1.png')
# poitdist = cv2.undistortPoints(np.float32(imgpo1),K1,dist1)
# poitdist.reshape(4,2).shape,np.float32(oj).shape

H2 = cv2.findHomography(np.float32(imgpo2),np.float32(oj),0)[0]


In [6]:
def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))
def linear_assignment(cost_matrix):
  try:
    import lap
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    return np.array([[y[i],i] for i in x if i >= 0]) #
  except ImportError:
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))


def iou_batch(bb_test, bb_gt):
  """
  From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
  """
  bb_gt = np.expand_dims(bb_gt, 0)
  bb_test = np.expand_dims(bb_test, 1)
  
  xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
  yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
  xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
  yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
  return(o)  


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))


def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))




def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

  iou_matrix = iou_batch(detections, trackers)

  if min(iou_matrix.shape) > 0:
    a = (iou_matrix > iou_threshold).astype(np.int32)
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


In [7]:
class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox,match=False):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    self.start_frame = 0
    KalmanBoxTracker.count += 1
    self.history = []
    self.history_pred = []
    self.history.append(convert_x_to_bbox(self.kf.x))
    self.hits = 0
    self.hit_streak = 0
    self.age = 0
    self.hasmatch = match

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
#     self.history = []
#     self.history_pred = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))
    self.history.append(convert_x_to_bbox(self.kf.x))
  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """

    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    tt = convert_x_to_bbox(self.kf.x)
    self.history_pred.append(tt)
    return tt

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)


In [20]:



class Sort(object):
  def __init__(self,H1,H2, max_age=1, min_hits=3, iou_threshold=0.2):
    """
    Sets key parameters for SORT
    """
    self.H1 = H1
    self.H2 = H2
    self.max_age = max_age
    self.min_hits = min_hits
    self.iou_threshold = iou_threshold
    self.trackers_first = []
    self.trackers_second = []
    self.trackers_del = []
    self.trackers_del_sec = []
    self.trks_ids_second = []
    self.trks_ids_first = []
    self.frame_count = 0
    self.ids_count = 0
    self.rem_track = 0
#     inds = self.get_pairs_match(det1,det2,H1,H2)
    
#   def intiate_tracks():
    
  def get_pairs_match(self,det_first, det_second,threshold = 70.470,switch=False):
    errors = []
    inds = []
   
    for i in range(det_first.shape[0]):
        errors = []
        for j in range(det_second.shape[0]):
            ss1,ss2=0,0
            pt1 = np.array([(det_first[i][0] + det_first[i][2])/2,det_first[i][3],1])
            if not switch:
                ss1 = np.matmul(self.H1,pt1)
            else:
                ss1 = np.matmul(self.H2,pt1)
            ss1 = ss1/ss1[2]
            pt2 = np.array([(det_second[j][0] + det_second[j][2])/2,det_second[j][3],1])
            if not switch:
                ss2 = np.matmul(self.H2,pt2)
            else:
                ss2 = np.matmul(self.H1,pt2)
            ss2 = ss2/ss2[2]
            er = math.sqrt(math.pow(ss1[0]-ss2[0],2) + math.pow(ss1[1]-ss2[1],2))
            errors.append(er)
        indo =  np.argsort(errors) 
        if indo.shape[0]==1 and errors[indo[0]]<threshold:
            inds.append((i,indo[0]))
        if indo.shape[0] >1 and errors[indo[0]] < .75*errors[indo[1]]:# and errors[indo[0]]<threshold:
            inds.append((i,indo[0]))
       
    return inds


        
  def update_stereo(self, dets_first=np.empty((0, 5)),dets_second=np.empty((0, 5)),frame_no=0):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    # get predicted locations from existing trackers.
    trks1 = np.zeros((len(self.trackers_first), 5))
    new_tracks = 0
    to_del = []
    ret = []
    for t, trk in enumerate(trks1):
      pos = self.trackers_first[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]#[true_pos[0], true_pos[1], true_pos[2], true_pos[3], 0] 
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks1 = np.ma.compress_rows(np.ma.masked_invalid(trks1))
 
    ###
    trks2 = np.zeros((len(self.trackers_second), 5))
    to_del = []
    ret = []
    for t, trk in enumerate(trks2):
      pos = self.trackers_second[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]#[true_pos[0], true_pos[1], true_pos[2], true_pos[3], 0]
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks2 = np.ma.compress_rows(np.ma.masked_invalid(trks2))
    matched1, unmatched_dets1, unmatched_trks1 = associate_detections_to_trackers(dets_first,trks1, self.iou_threshold)
    matched2, unmatched_dets2, unmatched_trks2 = associate_detections_to_trackers(dets_second,trks2, self.iou_threshold)
    
    if len(trks1)>0 and len(unmatched_dets2)>0: 
        inds1 = self.get_pairs_match(dets_second[unmatched_dets2],trks1)
        for i,j in inds1:
             trk = KalmanBoxTracker(dets_second[unmatched_dets2[i],:],match=True)
             trk.start_frame = frame_no
             trk.id = self.trks_ids_first[j]
             self.trackers_second.append(trk)
             self.trks_ids_second.append(self.trks_ids_first[j])
             new_tracks+=1
             pt2 = np.array([(dets_second[unmatched_dets2[i],:][0] + dets_second[unmatched_dets2[i],:][2])/2,dets_second[unmatched_dets2[i],:][3],1])
        if len(inds1)>0:
            unmatched_dets2 = np.delete(unmatched_dets2,[i for i,j in inds1 ],0) 
            
    if len(trks2)>0 and len(unmatched_dets1)>0:
        inds2 = self.get_pairs_match(dets_first[unmatched_dets1],trks2)
        for i,j in inds2:
             trk = KalmanBoxTracker(dets_first[unmatched_dets1[i],:],match=True)
             trk.start_frame = frame_no
             trk.id = self.trks_ids_second[j]
             self.trackers_first.append(trk)
             self.trks_ids_first.append(self.trks_ids_second[j]) 
             new_tracks+=1
             pt2 = np.array([(dets_first[unmatched_dets1[i],:][0] + dets_first[unmatched_dets1[i],:][2])/2,dets_first[unmatched_dets1[i],:][3],1])
        if len(inds2)>0:
            unmatched_dets1 = np.delete(unmatched_dets1,[i for i,j in inds2 ],0) 

    inds3 = []
    if len(unmatched_dets2)>0 and len(unmatched_dets1)>0:
        inds3 = self.get_pairs_match(dets_first[unmatched_dets1],dets_second[unmatched_dets2])
    for i,j in inds3:
        trk = KalmanBoxTracker(dets_first[unmatched_dets1[i],:],match=True)
        trk.start_frame = frame_no
        trk.id = self.ids_count
        self.trackers_first.append(trk)
        
        trk = KalmanBoxTracker(dets_second[unmatched_dets2[j],:],match=True)
        trk.start_frame = frame_no
        trk.id = self.ids_count
        self.trackers_second.append(trk)   
        
        self.trks_ids_first.append(self.ids_count)
        self.trks_ids_second.append(self.ids_count)
        self.ids_count+=1
        new_tracks+=1
    match_inds1  = [t for t,v in inds3 ]
    for i in unmatched_dets1:
        if i not in match_inds1:
            trk = KalmanBoxTracker(dets_first[i,:])
            trk.start_frame = frame_no
            trk.id = self.ids_count
            self.trackers_first.append(trk) 
            self.trks_ids_first.append(self.ids_count)
            self.ids_count+=1
            new_tracks+=1
    match_inds2   = [v for t,v in inds3 ]
    for i in unmatched_dets2:
        if i not in match_inds2:
            trk = KalmanBoxTracker(dets_second[i,:])
            trk.start_frame = frame_no
            trk.id = self.ids_count
            self.trackers_second.append(trk)  
            self.trks_ids_second.append(self.ids_count)
            self.ids_count+=1
            new_tracks+=1
            
    for m in matched1:
          self.trackers_first[m[1]].update(dets_first[m[0], :])
    for m in matched2:
          self.trackers_second[m[1]].update(dets_second[m[0], :])
            
    i = len(self.trackers_first)
    for trk in reversed(self.trackers_first):
        i -= 1
        # remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers_del.append(self.trackers_first[i])
          self.trackers_first.pop(i)
          self.trks_ids_first.pop(i) 
          
          self.rem_track+=1
    i = len(self.trackers_second)
    for trk in reversed(self.trackers_second):
        i -= 1
        if(trk.time_since_update > self.max_age):
          self.trackers_del_sec.append(self.trackers_second[i])
          self.trackers_second.pop(i)
          self.trks_ids_second.pop(i)  
          self.rem_track+=1
    if(len(ret)>0):
      return np.concatenate(ret)

    return np.empty((0,5))

class Args():
    def __init__(self):
        self.max_age= 30
        self.min_hits=3
        self.iou_threshold=.3
        self.phase='train'
        self.seq_path = 'cam.txt'
        self.display = False


In [21]:
args = Args()
display = args.display
phase = args.phase
total_time = 0.0
total_frames = 0
seq_dets_first = np.loadtxt('cam1_basket1.txt', delimiter=' ')
seq_dets_second = np.loadtxt('cam2_basket2.txt', delimiter=' ')
mot_tracker = Sort(H1,H2,max_age=args.max_age, 
                       min_hits=args.min_hits,
                       iou_threshold=args.iou_threshold)

for frame in range(int(seq_dets_first[:,0].max())):
#     frame += 0 #detection and frame numbers begin at 1
    dets_first = seq_dets_first[seq_dets_first[:, 0]==frame, 2:7]
    #     print(dets_first)
    #     dets_first[:, 2:4] += dets_first[:, 0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
    dets_second = seq_dets_second[seq_dets_second[:, 0]==frame, 2:7]
    #     dets_second[:, 2:4] += dets_second[:, 0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
    #     total_frames += 1
    mot_tracker.update_stereo(dets_first,dets_second,frame)
    #trying with 15 second only
    if frame==30*15:
        break

In [128]:
import numpy as np
import skvideo.io
img = cv2.imread('frame0.png')
vcap1 = cv2.VideoCapture(r"C:\Users\ahmed.mansour\Downloads\vid1.mp4")
out_video = None
frame_no = 14*30
vcap1.set(1,frame_no)
img_num =0
while(1):
    ret, img1 = vcap1.read()
    if out_video is  None:
        out_video =  np.empty([30*15,img1.shape[0],img1.shape[1], 3], dtype = np.uint8)
        out_video =  out_video.astype(np.uint8)
    for i in range(len(mot_tracker.trackers_first)):
      max_ind = len(mot_tracker.trackers_first[i].history)+ mot_tracker.trackers_first[i].start_frame
      if img_num>=mot_tracker.trackers_first[i].start_frame \
        and max_ind > img_num:
          pt1 = mot_tracker.trackers_first[i].history[img_num-mot_tracker.trackers_first[i].start_frame][0]
          img1 = cv2.rectangle(img1,(int(pt1[0]),int(pt1[1])),(int(pt1[2]),int(pt1[3])), (255,0,0))
          img1 = cv2.putText(img1, str(mot_tracker.trks_ids_first[i]), (int(pt1[0]),int(pt1[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
    
    for i in range(len(mot_tracker.trackers_del)):
      max_ind = len(mot_tracker.trackers_del[i].history)+ mot_tracker.trackers_del[i].start_frame
      if img_num>=mot_tracker.trackers_del[i].start_frame \
        and max_ind > img_num:
          pt1 = mot_tracker.trackers_del[i].history[img_num-mot_tracker.trackers_del[i].start_frame][0]
          img1 = cv2.rectangle(img1,(int(pt1[0]),int(pt1[1])),(int(pt1[2]),int(pt1[3])), (255,0,0))
          img1 = cv2.putText(img1, str(mot_tracker.trackers_del[i].id), (int(pt1[0]),int(pt1[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
    out_video[img_num] = img1
    img_num+=1
    if img_num==30*15:
        break

# Writes the the output image sequences in a video file
skvideo.io.vwrite("video_no10.mp4", out_video)

In [34]:
import numpy as np
import skvideo.io
vcap1 = cv2.VideoCapture(r"C:\Users\ahmed.mansour\Downloads\video22.mp4")
out_video = None
frame_no = (3*60+43)*30
vcap1.set(1,frame_no)
img_num =0
while(1):
    ret, img1 = vcap1.read()
    if out_video is  None:
        out_video =  np.empty([30*15,img1.shape[0],img1.shape[1], 3], dtype = np.uint8)
        out_video =  out_video.astype(np.uint8)
    for i in range(len(mot_tracker.trackers_second)):
      max_ind = len(mot_tracker.trackers_second[i].history)+ mot_tracker.trackers_second[i].start_frame
      if img_num>=mot_tracker.trackers_second[i].start_frame \
        and max_ind > img_num:
          pt1 = mot_tracker.trackers_second[i].history[img_num-mot_tracker.trackers_second[i].start_frame][0]
          img1 = cv2.rectangle(img1,(int(pt1[0]),int(pt1[1])),(int(pt1[2]),int(pt1[3])), (255,0,0))
          img1 = cv2.putText(img1, str(mot_tracker.trks_ids_second[i]), (int(pt1[0]),int(pt1[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
    out_video[img_num] = img1
    img_num+=1
    if img_num==30*15:
        break

# Writes the the output image sequences in a video file
skvideo.io.vwrite("video_1_n10.mp4", out_video)

In [22]:
#detctions_tracking
import numpy as np
import skvideo.io
vcap1 = cv2.VideoCapture(r"C:\Users\ahmed.mansour\Downloads\videoreal.mp4")
out_video = None
frame_no = 0#14*30
vcap1.set(1,frame_no)
img_num =0
while(1):
    ret, img1 = vcap1.read()
    if out_video is  None:
        out_video =  np.empty([30*15,img1.shape[0],img1.shape[1], 3], dtype = np.uint8)
        out_video =  out_video.astype(np.uint8)
    for i in range(len(mot_tracker.trackers_first)):
      max_ind = len(mot_tracker.trackers_first[i].history_pred) + mot_tracker.trackers_first[i].start_frame
      if img_num>=mot_tracker.trackers_first[i].start_frame \
        and max_ind > img_num:
          pt1 = mot_tracker.trackers_first[i].history_pred[img_num-mot_tracker.trackers_first[i].start_frame][0]
          img1 = cv2.rectangle(img1,(int(pt1[0]),int(pt1[1])),(int(pt1[2]),int(pt1[3])), (255,165,0),2)
          img1 = cv2.putText(img1, str(mot_tracker.trackers_first[i].id), (int(pt1[0]),int(pt1[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,165,0), 3)
    for i in range(len(mot_tracker.trackers_del)):
      max_ind = len(mot_tracker.trackers_del[i].history_pred) + mot_tracker.trackers_del[i].start_frame
      if img_num>=mot_tracker.trackers_del[i].start_frame \
        and max_ind > img_num:
          pt1 = mot_tracker.trackers_del[i].history_pred[img_num-mot_tracker.trackers_del[i].start_frame][0]
          img1 = cv2.rectangle(img1,(int(pt1[0]),int(pt1[1])),(int(pt1[2]),int(pt1[3])), (255,165,0),3)
          img1 = cv2.putText(img1, str(mot_tracker.trackers_del[i].id), (int(pt1[0]),int(pt1[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,165,0), 3)
    out_video[img_num] = img1
    img_num+=1
    if img_num==30*15:
        break

# Writes the the output image sequences in a video file
skvideo.io.vwrite("video_real_tracking.mp4", out_video)

In [25]:
#detctions_tracking
import numpy as np
import skvideo.io
img = cv2.imread('frame0.png')
vcap1 = cv2.VideoCapture(r"C:\Users\ahmed.mansour\Downloads\videoreal2.mp4")
out_video = None
frame_no = 0#14*30
vcap1.set(1,frame_no)
img_num =0
while(1):
    ret, img1 = vcap1.read()
    if out_video is  None:
        out_video =  np.empty([30*15,img1.shape[0],img1.shape[1], 3], dtype = np.uint8)
        out_video =  out_video.astype(np.uint8)
    for i in range(len(mot_tracker.trackers_second)):
      max_ind = len(mot_tracker.trackers_second[i].history_pred) + mot_tracker.trackers_second[i].start_frame 
      if img_num>=mot_tracker.trackers_second[i].start_frame \
        and max_ind > img_num:
          pt1 = mot_tracker.trackers_second[i].history_pred[img_num-mot_tracker.trackers_second[i].start_frame][0]
          img1 = cv2.rectangle(img1,(int(pt1[0]),int(pt1[1])),(int(pt1[2]),int(pt1[3])), (255, 165, 0))
          img1 = cv2.putText(img1, str(mot_tracker.trackers_second[i].id), (int(pt1[0]),int(pt1[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 165, 0), 3)
    for i in range(len(mot_tracker.trackers_del_sec)):
      max_ind = len(mot_tracker.trackers_del_sec[i].history_pred) + mot_tracker.trackers_del_sec[i].start_frame
      if img_num>=mot_tracker.trackers_del_sec[i].start_frame \
        and max_ind > img_num:
          pt1 = mot_tracker.trackers_del_sec[i].history_pred[img_num-mot_tracker.trackers_del_sec[i].start_frame][0]
          img1 = cv2.rectangle(img1,(int(pt1[0]),int(pt1[1])),(int(pt1[2]),int(pt1[3])), (255, 165, 0))
          img1 = cv2.putText(img1, str(mot_tracker.trackers_del_sec[i].id), (int(pt1[0]),int(pt1[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 165, 0), 3)
    out_video[img_num] = img1
    img_num+=1
    if img_num==30*15:
        break

# Writes the the output image sequences in a video file
skvideo.io.vwrite("video_real_tracking2.mp4", out_video)